# Pure TF-IDF Model

### Imports

In [1]:
% reset
import numpy as np
import os
import nltk
import re
import ntpath
import pathlib

from helpers.cleaner_TFIDF import *

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from gensim.corpora import Dictionary

from multiprocessing import Process,Lock,Value

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier,LogisticRegressionCV,LogisticRegression
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,BaggingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split,GridSearchCV
import sklearn.grid_search
from sklearn import cross_validation

%load_ext autoreload
%autoreload 2

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


/home/mohamedmahmoud/myprograms/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/mohamedmahmoud/myprograms/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
ntpath.basename("a/b/c/d/e")

def path_leaf(path):
    a,b,c,d,e = path.split("/")
    return d,e

In [3]:
dirs_train = ['Data/aclImdb/train/pos','Data/aclImdb/train/neg']
clean_root_train = speedy(dirs_train,15)
dirs_test = ['Data/aclImdb/test/pos','Data/aclImdb/test/neg']
clean_root_test = speedy(dirs_test,15)

In [4]:
from pathlib import Path
class MyDocs(object):
    def __init__(self, dirname):
        self.dirname = dirname
        self.pos = []
        self.neg = []
        self.loopy()
 
    def __iter__(self):
        self.loopy()
    
    def loopy(self):
        pathlist = Path(self.dirname).glob('**/*')
        for path in pathlist:
            path_in_str = str(path)
            if os.path.isfile(path_in_str):
                f=open(path_in_str)
                parent, uid = path_leaf(path_in_str)
                uid, _ = uid.split("_")
                uid = int(str(uid))
                if parent == "pos":
                    self.pos.append(f.read())
                elif parent == "neg":
                    self.neg.append(f.read())

In [5]:
docs_train = MyDocs(clean_root_train)

In [6]:
pos_train = np.array(docs_train.pos)
neg_train = np.array(docs_train.neg)
X_train = np.concatenate((pos_train,neg_train),axis=0)
y_train = np.asarray([1]*pos_train.shape[0] + [-1]*neg_train.shape[0])

### Hyperparameter tuning and crossvalidating the training set

In [10]:
X_train,X_val,y_train,y_val = cross_validation.train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [11]:
pipeliner = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),('clf', LogisticRegression(solver='lbfgs')),])

In [43]:
pg1 = [
    {
        'clf': [MultinomialNB()],
        'clf__alpha': [0.5,1]
    }
]

In [44]:
nb = GridSearchCV(pipeliner, param_grid=pg1, refit=True)

In [45]:
nb.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...enalty='l2', random_state=None, solver='lbfgs', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'clf': [MultinomialNB(alpha=0.5, class_prior=None, fit_prior=True)], 'clf__alpha': [0.5, 1]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [46]:
best1 = nb.best_estimator_
print("Best estimator found:")
print(best1)

print("Best score:")
print(nb.best_score_)

print("Best parameters found:")
print(nb.best_params_)

Best estimator found:
Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=0.5, class_prior=None, fit_prior=True))])
Best score:
0.8616
Best parameters found:
{'clf': MultinomialNB(alpha=0.5, class_prior=None, fit_prior=True), 'clf__alpha': 0.5}


In [15]:
pg2 = [
    {
        'clf': [AdaBoostClassifier()],
        'clf__n_estimators': [50,100,200]
        'clf__n_jobs': [-1]
    }
]

In [16]:
adaboost = GridSearchCV(pipeliner, param_grid=pg2, refit=True)

In [17]:
adaboost.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...enalty='l2', random_state=None, solver='lbfgs', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'clf': [AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=200, random_state=None)], 'clf__n_estimators': [50, 100, 200]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [18]:
best2 = adaboost.best_estimator_
print("Best estimator found:")
print(best2)

print("Best score:")
print(adaboost.best_score_)

print("Best parameters found:")
print(adaboost.best_params_)

Best estimator found:
Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=200, random_state=None))])
Best score:
0.845355555556
Best parameters found:
{'clf': AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=200, random_state=None), 'clf__n_estimators': 200}


In [19]:
pg3 = [
    {
        'clf': [RandomForestClassifier()],
        'clf__n_estimators': [50,100,200],
        'clf__n_jobs': [-1]
    }
]

In [20]:
forest = GridSearchCV(pipeliner, param_grid=pg3, refit=True)

In [21]:
forest.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...enalty='l2', random_state=None, solver='lbfgs', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'clf': [RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_...erbose=0,
            warm_start=False)], 'clf__n_estimators': [50, 100, 200], 'clf__n_jobs': [-1]}],
       pre_dispatch='2*n_jobs', refit=True, return_

In [22]:
best3 = forest.best_estimator_
print("Best estimator found:")
print(best3)

print("Best score:")
print(forest.best_score_)

print("Best parameters found:")
print(forest.best_params_)

Best estimator found:
Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip..._jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])
Best score:
0.855333333333
Best parameters found:
{'clf': RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False), 'clf__n_estimators': 200, 'clf__n_jobs': -1}


In [53]:
pg4 = [
    {
        'clf': [SVC()],
        'clf__C': [0.1, 1, 5],
        
    }
]

In [54]:
svc = GridSearchCV(pipeliner, param_grid=pg4, refit=True)

In [60]:
svc.fit(X_train, y_train)

In [ ]:
best4 = svc.best_estimator_
print("Best estimator found:")
print(best4)

print("Best score:")
print(svc.best_score_)

print("Best parameters found:")
print(svc.best_params_)

In [56]:
pg5 = [
    {
        'clf': [DecisionTreeClassifier()],
        'clf__min_samples_split': [2, 10, 20]
    }
]

In [57]:
tree = GridSearchCV(pipeliner, param_grid=pg5, refit=True)

In [58]:
tree.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...enalty='l2', random_state=None, solver='lbfgs', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'clf': [DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')], 'clf__min_samples_split': [2, 10, 20]}],
       pre_dispatch='2*n_jobs', refit=True

In [59]:
best5 = tree.best_estimator_
print("Best estimator found:")
print(best5)

print("Best score:")
print(tree.best_score_)

print("Best parameters found:")
print(tree.best_params_)

Best estimator found:
Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...      min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'))])
Best score:
0.715466666667
Best parameters found:
{'clf': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'clf__min_samples_split': 2}


In [27]:
pg6 = [
    {
        'clf': [LogisticRegression()],
        'clf__C': [0.1,5,10,15,20]
    }
]

In [28]:
logistic = GridSearchCV(pipeliner, param_grid=pg6, refit=True)

In [29]:
logistic.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...enalty='l2', random_state=None, solver='lbfgs', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'clf': [LogisticRegression(C=5, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)], 'clf__C': [0.1, 5, 10, 15, 20]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [30]:
best6 = logistic.best_estimator_
print("Best estimator found:")
print(best6)

print("Best score:")
print(logistic.best_score_)

print("Best parameters found:")
print(logistic.best_params_)

Best estimator found:
Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])
Best score:
0.892466666667
Best parameters found:
{'clf': LogisticRegression(C=5, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False), 'clf__C': 5}


In [7]:
docs_test = MyDocs(clean_root_test)

pos_test = np.array(docs_test.pos)
neg_test = np.array(docs_test.neg)
X_test = np.concatenate((pos_test,neg_test),axis=0)
y_test = np.asarray([1]*pos_train.shape[0] + [-1]*neg_train.shape[0])

In [ ]:
# y_pred = cv.predict(X_test)
# print(classification_report(y_test, y_pred))

### Classifiers

In [8]:
text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),('clf', MultinomialNB(alpha=0.5)),])

In [9]:
text_clf.fit(X_train, y_train)
text_clf.score(X_test,y_test)

0.90803999999999996

In [11]:
text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),('clf', AdaBoostClassifier(n_estimators=200)),])

In [12]:
text_clf.fit(X_train, y_train)
text_clf.score(X_test,y_test)

0.85850000000000004

In [18]:
text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),('clf', SVC(C=1)),])

In [20]:
text_clf.fit(X_train, y_train)
text_clf.score(X_test,y_test)

In [21]:
text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),('clf', DecisionTreeClassifier(min_samples_split=2)),])

In [22]:
text_clf.fit(X_train, y_train)
text_clf.score(X_test,y_test)

1.0

In [21]:
text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),('clf', LogisticRegression(solver='lbfgs')),])

In [22]:
text_clf.fit(X_train, y_train)
text_clf.score(X_test,y_test)

0.92874000000000001

In [23]:
text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),('clf', LogisticRegressionCV(solver='lbfgs')),])

In [24]:
text_clf.fit(X_train, y_train)
text_clf.score(X_test,y_test)

0.95098000000000005